## Apply Real-ESRGAN over the satellite imagery dataset for semantic segmentation


---

### Import Libs

In [4]:
import torch
import torch.utils.data as data

from torchvision.transforms import ToTensor

from sklearn.preprocessing import MinMaxScaler

import imageio.v2 as imageio

import rasterio
from PIL import Image

import random
import numpy as np
import glob
import re
import sys

import os
import subprocess
from tqdm import tqdm

from datetime import datetime

import cv2


In [5]:
# Set the Multiprocessing Start Method for Windows
import torch.multiprocessing as mp

if __name__ == "__main__":
    mp.set_start_method('spawn', force=True)

# Set a known seed
seed = 23
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

### CUDA Configuration

In [7]:
print(70*'-')
print("CUDA available:", torch.cuda.is_available())
current_device = torch.cuda.current_device()
print("GPU Model:", torch.cuda.get_device_name(current_device))
print("CUDA Version:", torch.version.cuda)
print(70*'-')

device_properties = torch.cuda.get_device_properties(current_device)
print("\nDevice Properties:")
print("  Name:", device_properties.name)
print("  Total Memory (GB):", device_properties.total_memory / (1024 ** 3))  # Convert bytes to GB
print("  Multiprocessors:", device_properties.multi_processor_count)
print("  Compute Capability:", device_properties.major, ".", device_properties.minor)
print(70*'-')

----------------------------------------------------------------------
CUDA available: True
GPU Model: NVIDIA GeForce GTX 1080 Ti
CUDA Version: 11.8
----------------------------------------------------------------------

Device Properties:
  Name: NVIDIA GeForce GTX 1080 Ti
  Total Memory (GB): 10.999755859375
  Multiprocessors: 28
  Compute Capability: 6 . 1
----------------------------------------------------------------------


### Data Access

In [12]:
# Function to extract the numeric value from filenames
def numeric_sort_key(filepath):
    # Extract numbers from the filename using a regular expression
    match = re.search(r'\d+', filepath)
    # Return the integer value of the number if found, otherwise 0
    return int(match.group()) if match else 0

# Get all the image and mask paths and sort them numerically
folder_data_train = sorted(
    glob.glob("../data/train/input/*tif"),
    key=numeric_sort_key
)
folder_mask_train = sorted(
    glob.glob("../data/train/labels/*tif"),
    key=numeric_sort_key
)

folder_data_val = sorted(
    glob.glob("../data/val/input/*tif"),
    key=numeric_sort_key
)
folder_mask_val = sorted(
    glob.glob("../data/val/labels/*tif"),
    key=numeric_sort_key
)

folder_data_test = sorted(
    glob.glob("../data/test/input/*tif"),
    key=numeric_sort_key
)
folder_mask_test = sorted(
    glob.glob("../data/test/labels/*tif"),
    key=numeric_sort_key
)

# Assign to variables
train_image_paths = folder_data_train[:]
val_image_paths = folder_data_val[:]
test_image_paths = folder_data_test[:]

train_mask_paths = folder_mask_train[:]
val_mask_paths = folder_mask_val[:]
test_mask_paths = folder_mask_test[:]



In [14]:
# Check the number of train, val and test images
print('\nControl number of images by folder:')
print(40*'=')
input_total = len(folder_data_train)+len(folder_data_val)+len(folder_data_test)
labels_total = len(folder_mask_train)+len(folder_mask_val)+len(folder_mask_test)
print('\t\tinput: \tlabels:')
print(f'Train\t\t{len(folder_data_train)}\t{len(folder_mask_train)}\t{(len(folder_data_train)/input_total)*100:.2f}%')
print(f'Val\t\t{len(folder_data_val)}\t{len(folder_mask_val)}\t{(len(folder_data_val)/input_total)*100:.2f}%')
print(f'Test\t\t{len(folder_data_test)}\t{len(folder_mask_test)}\t{(len(folder_data_test)/input_total)*100:.2f}%')
print(40*'=')
print(f'Total\t\t{input_total}\t{labels_total}')


Control number of images by folder:
		input: 	labels:
Train		201	201	73.90%
Val		35	35	12.87%
Test		36	36	13.24%
Total		272	272


### Custom Dataset

In [17]:
class CustomDataset(data.Dataset):
    def __init__(self, image_paths, target_paths=None, transform=None, band=None, device="cuda"):
        self.image_paths = image_paths
        self.target_paths = target_paths
        self.transform = transform
        self.band = band  # Specify which band to use (0: R, 1: G, 2: B, 3: NIR, None: all bands)
        self.scaler = MinMaxScaler()
        self.device = device

    def __getitem__(self, index):
        try:
            # Load the GeoTIFF image
            image_path = self.image_paths[index]
            with rasterio.open(image_path) as src:
                if self.band is not None:
                    # Read a specific band (1-based indexing for rasterio)
                    band_data = src.read(self.band + 1).astype(np.float32)
                    band_min, band_max = band_data.min(), band_data.max()
                    band_data = (band_data - band_min) / (band_max - band_min + 1e-8)
                    image = band_data[None, :, :]  # Add channel dimension
                else:
                    # Read all bands and normalize each
                    bands = src.read().astype(np.float32)  # [Bands, H, W]
                    image = np.zeros_like(bands, dtype=np.float32)
                    for b in range(bands.shape[0]):
                        band_min, band_max = bands[b].min(), bands[b].max()
                        image[b] = (bands[b] - band_min) / (band_max - band_min + 1e-8)

            # Ensure the image is a NumPy array for transformations
            image = np.transpose(image, (1, 2, 0))  # [H, W, C] for compatibility with PIL
            pil_image = Image.fromarray((image * 255).astype(np.uint8))  # Convert to PIL Image

            # Apply transformations if specified
            if self.transform:
                image = self.transform(pil_image)  # Transform will handle conversion to PyTorch Tensor

            # Load the mask if provided
            if self.target_paths:
                mask_path = self.target_paths[index]
                with rasterio.open(mask_path) as src:
                    mask_data = src.read(1).astype(np.uint8)  # Single-channel mask

                # Resize the mask using nearest-neighbor interpolation
                mask_image = Image.fromarray(mask_data)
                mask_image = mask_image.resize(pil_image.size, resample=Image.NEAREST)
                mask_data = np.array(mask_image, dtype=np.uint8)
                mask_data = torch.tensor(mask_data[None, :, :], dtype=torch.float32)  # [1, H, W]
            else:
                mask_data = None

            return image, mask_data

        except Exception as e:
            print(f"Error processing index {index}: {e}")
            raise e

    def __len__(self):
        return len(self.image_paths)



### Unitary test

In [20]:
from torchvision.transforms import ToTensor

# Paths to images and masks
image_paths = ["../data/train/input/img(21).tif"]
mask_paths = ["../data/train/labels/target(21).tif"]

# Create dataset
dataset = CustomDataset(
    image_paths=image_paths,
    target_paths=mask_paths,
    transform=ToTensor(),
    band=None  # None to load all bands
)

# Access an item
image, mask = dataset[0]
print(f"Image shape: {image.shape}, dtype: {image.dtype}")  # Expected: [C, H, W]
print(f"Mask shape: {mask.shape}, dtype: {mask.dtype}")    # Expected: [1, H, W]



Image shape: torch.Size([4, 256, 256]), dtype: torch.float32
Mask shape: torch.Size([1, 256, 256]), dtype: torch.float32


In [22]:
# Initialize the CustomDataset objects
use_all_bands = True
model_band = None # 0:red, 1:green, 2:blue, 3:NIR, None:all 4 bands

train_dataset = CustomDataset(train_image_paths, train_mask_paths, transform=ToTensor(), band=model_band)
val_dataset = CustomDataset(val_image_paths, val_mask_paths, transform=ToTensor(), band=model_band)
test_dataset = CustomDataset(test_image_paths, test_mask_paths, transform=ToTensor(), band=model_band)

image, mask = train_dataset[0]  # Load the first item
print(f"Transformed image shape: {image.shape}, dtype: {image.dtype}")
print(f"Transformed mask shape: {mask.shape}, dtype: {mask.dtype}")

Transformed image shape: torch.Size([4, 256, 256]), dtype: torch.float32
Transformed mask shape: torch.Size([1, 256, 256]), dtype: torch.float32


---

# Transformation 


## Step 1 - Unpack GeoTIFF Files

In [40]:

def unpack_tiff_files(data_root):
    """
    Unpacks GeoTIFF files into individual band images and binary masks,
    following the specified folder structure.
    
    Args:
        data_root (str): Path to the dataset root folder.
    """
    subsets = ["train", "val", "test"]

    for subset in subsets:
        input_dir = os.path.join(data_root, subset, "input")
        label_dir = os.path.join(data_root, subset, "labels")

        print(f"Processing subset: {subset}")
        # Process input images
        for filename in tqdm(os.listdir(input_dir)):
            if filename.endswith(".tif"):
                input_path = os.path.join(input_dir, filename)
                base_name, _ = os.path.splitext(filename)

                with rasterio.open(input_path) as src:
                    # Read all bands
                    bands = src.read()
                    if bands.shape[0] != 4:
                        raise ValueError(f"Expected 4 bands in {filename}, got {bands.shape[0]}")

                    # Save each band as a separate PNG
                    band_names = ['r', 'g', 'b', 'n']
                    for i, band_name in enumerate(band_names):
                        output_path = os.path.join(input_dir, f"{base_name}_{band_name}.png")
                        band_data = bands[i]
                        band_image = Image.fromarray((band_data / band_data.max() * 255).astype(np.uint8))
                        band_image.save(output_path)

        # Process label masks
        for filename in tqdm(os.listdir(label_dir)):
            if filename.endswith(".tif"):
                label_path = os.path.join(label_dir, filename)
                base_name, _ = os.path.splitext(filename)

                with rasterio.open(label_path) as src:
                    # Read the single-band mask
                    mask = src.read(1)  # Assuming the mask is in the first band
                    output_path = os.path.join(label_dir, f"{base_name}_b.png")
                    mask_image = Image.fromarray((mask / mask.max() * 255).astype(np.uint8))
                    mask_image.save(output_path)

# Define the dataset root path
data_root = "../data"

# Call the function
unpack_tiff_files(data_root)


Processing subset: train


  0%|                                                                                          | 0/402 [00:00<?, ?it/s]C:\Users\fpere\AppData\Local\Temp\ipykernel_8932\3040735809.py:52: RuntimeWarning: invalid value encountered in divide
  mask_image = Image.fromarray((mask / mask.max() * 255).astype(np.uint8))
C:\Users\fpere\AppData\Local\Temp\ipykernel_8932\3040735809.py:52: RuntimeWarning: invalid value encountered in cast
  mask_image = Image.fromarray((mask / mask.max() * 255).astype(np.uint8))
100%|███████████████████████████████████████████████████████████████████████████████| 402/402 [00:01<00:00, 256.74it/s]


Processing subset: val


100%|█████████████████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 356.28it/s]


Processing subset: test


100%|█████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 359.17it/s]


## Step 2 - Transformation phase
### input band images -> Super Resolution | binary mask images -> Nearest Neighbor

In [28]:
data_root = "../data"
results_root = "../results_x4/"
model_name = "RealESRGAN_x4plus"

# Ensure results directories exist
os.makedirs(results_root, exist_ok=True)

# Loop through train, val, and test directories
for phase in ["train", "val", "test"]:
    input_dir = os.path.join(data_root, phase, "input")
    output_dir = os.path.join(results_root, phase, "input")
    os.makedirs(output_dir, exist_ok=True)

    # Process all PNG images in the input folder
    for file_name in tqdm(os.listdir(input_dir), desc=f"Processing {phase} inputs"):
        if file_name.endswith(".png"):
            input_file = os.path.join(input_dir, file_name)
            output_file = os.path.join(output_dir, file_name)

            # Run the Real-ESRGAN script
            python_path = r"E:\Devs\pyEnv-1\Real-ESRGAN_Experimental\Real-ESRGAN_env\Scripts\python.exe"
            command = [
                python_path, "inference_realesrgan.py",
                "-n", model_name,
                "-i", input_file,
                "-o", output_dir,
                "--outscale", "4",
                "--suffix", ""
            ]

            try:
                # Capture output and error
                result = subprocess.run(command, check=True, text=True, capture_output=True)
                print(f"Success for {file_name}:\n{result.stdout}")
            except subprocess.CalledProcessError as e:
                print(f"Error for {file_name}:\n{e.stderr}")
                raise e



Processing train inputs:   0%|                                                      | 2/1006 [00:07<1:01:13,  3.66s/it]

Success for img(100)_b.png:
Testing 0 img(100)_b



Processing train inputs:   0%|▏                                                     | 3/1006 [00:13<1:19:37,  4.76s/it]

Success for img(100)_g.png:
Testing 0 img(100)_g



Processing train inputs:   0%|▏                                                     | 4/1006 [00:19<1:27:38,  5.25s/it]

...

Success for img(9)_g.png:
Testing 0 img(9)_g



Processing test inputs:  99%|███████████████████████████████████████████████████████▋| 179/180 [14:05<00:04,  4.91s/it]

Success for img(9)_n.png:
Testing 0 img(9)_n



Processing test inputs: 100%|████████████████████████████████████████████████████████| 180/180 [14:11<00:00,  4.73s/it]

Success for img(9)_r.png:
Testing 0 img(9)_r



### Apply the transformation for the mask files using Nearest Neighbor with Morphological Smoothing

Why Use Morphological Smoothing?

When we upscale a binary mask (using nearest neighbor), the edges become jagged or "blocky." This happens because Nearest Neighbor interpolation simply replicates pixel values, leading to stair-step patterns along diagonal lines. Morphological smoothing:

- Removes these sharp edges.
- Ensures the lines and shapes in your mask are more natural and consistent.
- Fills in small gaps while maintaining the binary (0 and 1) nature of your mask.

In [43]:

# Function to apply nearest neighbor resizing with morphological smoothing
def resize_mask_with_smoothing(mask_path, reference_path, output_path):
    """
    Resize mask to match the resolution of the reference image using nearest neighbor interpolation
    and apply morphological smoothing.
    """
    # Open the mask
    with rasterio.open(mask_path) as src:
        mask = src.read(1)  # Read the first (and only) band

    # Open the reference image
    with Image.open(reference_path) as ref_img:
        target_size = ref_img.size  # (width, height)

    # Resize the mask
    mask_resized = Image.fromarray(mask).resize(target_size, resample=Image.NEAREST)

    # Convert the resized mask to a NumPy array
    mask_resized_np = np.array(mask_resized)

    # Apply morphological smoothing (closing: dilation followed by erosion)
    kernel = np.ones((3, 3), np.uint8)  # Define the kernel size
    smoothed_mask = cv2.morphologyEx(mask_resized_np, cv2.MORPH_CLOSE, kernel)

    # Convert back to binary (ensure only 0 and 1 values)
    smoothed_mask_binary = (smoothed_mask > 127).astype(np.uint8) * 255  # Scale to 0 and 255

    # Save the resized and smoothed mask
    smoothed_image = Image.fromarray(smoothed_mask_binary)
    smoothed_image.save(output_path, format="PNG")

# Process labels for train, val, and test
for phase in ["train", "val", "test"]:
    labels_dir = os.path.join(data_root, phase, "labels")
    output_labels_dir = os.path.join(results_root, phase, "labels")
    input_dir = os.path.join(results_root, phase, "input")
    os.makedirs(output_labels_dir, exist_ok=True)

    for file_name in tqdm(os.listdir(labels_dir), desc=f"Processing {phase} labels"):
        if file_name.endswith("_b.png"):
            mask_path = os.path.join(labels_dir, file_name)
            file_name_replace = re.sub(r'^.*?\(', 'img(', file_name)
            reference_path = os.path.join(input_dir, file_name_replace.replace("_b", "_r"))  # Match with the red band
            output_path = os.path.join(output_labels_dir, file_name)

            resize_mask_with_smoothing(mask_path, reference_path, output_path)




# # Function to apply nearest neighbor resizing
# def resize_mask(mask_path, reference_path, output_path):
#     """
#     Resize mask to match the resolution of the reference image using nearest neighbor interpolation.
#     """
#     # Open the mask
#     with rasterio.open(mask_path) as src:
#         mask = src.read(1)  # Read the first (and only) band

#     # Open the reference image
#     with Image.open(reference_path) as ref_img:
#         target_size = ref_img.size  # (width, height)

#     # Resize the mask
#     mask_resized = Image.fromarray(mask).resize(target_size, resample=Image.NEAREST)

#     # Save the resized mask
#     mask_resized.save(output_path, format="PNG")

# # Process labels for train, val, and test
# for phase in ["train", "val", "test"]:
#     labels_dir = os.path.join(data_root, phase, "labels")
#     output_labels_dir = os.path.join(results_root, phase, "labels")
#     input_dir = os.path.join(results_root, phase, "input")
#     os.makedirs(output_labels_dir, exist_ok=True)

#     for file_name in tqdm(os.listdir(labels_dir), desc=f"Processing {phase} labels"):
#         if file_name.endswith("_b.png"):
#             mask_path = os.path.join(labels_dir, file_name)
#             file_name_replace = re.sub(r'^.*?\(', 'img(', file_name)
#             reference_path = os.path.join(input_dir, file_name_replace.replace("_b", "_r"))  # Match with the red band
#             output_path = os.path.join(output_labels_dir, file_name)

#             resize_mask(mask_path, reference_path, output_path)


Processing test labels: 100%|█████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 167.11it/s]


## Step 3 - Pack the new GeoTIFF files

In [45]:

data_root = "../data/"
results_root = "../results_x4/"

from rasterio.transform import Affine

def pack_geotiff(input_metadata, output_path, bands_data, scale_factor, is_mask=False):
    """
    Packs the bands_data into a new GeoTIFF file using the original metadata.
    
    Args:
        input_metadata: Metadata from the original GeoTIFF file.
        output_path: Path to the new GeoTIFF file.
        bands_data: List of numpy arrays representing the bands.
        scale_factor: Upscaling factor applied to the bands.
        is_mask: Boolean indicating if the data corresponds to a mask (for binary transformation).
    """
    # Adjust the transform to maintain the same geographic area
    original_transform = input_metadata["transform"]
    new_transform = Affine(
        original_transform.a / scale_factor,
        original_transform.b,
        original_transform.c,
        original_transform.d,
        original_transform.e / scale_factor,
        original_transform.f
    )
    input_metadata.update({
        "height": bands_data[0].shape[0],
        "width": bands_data[0].shape[1],
        "transform": new_transform
    })

    # Apply transformation for masks (convert 255 values to 1)
    if is_mask:
        bands_data = [np.where(band == 255, 1, band).astype(np.float32) for band in bands_data]

    with rasterio.open(
        output_path, "w", 
        driver="GTiff",
        height=bands_data[0].shape[0],
        width=bands_data[0].shape[1],
        count=len(bands_data),  # Number of bands
        dtype=bands_data[0].dtype,
        crs=input_metadata["crs"],
        transform=new_transform
    ) as dst:
        for idx, band in enumerate(bands_data, start=1):
            dst.write(band, idx)

# Process the datasets
scale_factor = 4  # The upscaling factor
for subset in ["train", "val", "test"]:
    for subfolder in ["input", "labels"]:
        print(f"Packing GeoTIFFs for {subset}/{subfolder}...")

        # Input folder paths
        data_folder = os.path.join(data_root, subset, subfolder)
        results_folder = os.path.join(results_root, subset, subfolder)

        # Process each GeoTIFF file in the data folder
        for file_name in tqdm(os.listdir(data_folder)):
            if not file_name.endswith(".tif"):
                continue

            # Paths
            original_tiff_path = os.path.join(data_folder, file_name)
            output_tiff_path = os.path.join(results_folder, file_name)

            # Read the original metadata
            with rasterio.open(original_tiff_path) as src:
                metadata = src.meta.copy()

            # Collect band data
            if subfolder == "input":
                # 4 bands: R, G, B, NIR
                bands = [
                    np.array(Image.open(os.path.join(results_folder, f"{file_name[:-4]}_r.png"))),
                    np.array(Image.open(os.path.join(results_folder, f"{file_name[:-4]}_g.png"))),
                    np.array(Image.open(os.path.join(results_folder, f"{file_name[:-4]}_b.png"))),
                    np.array(Image.open(os.path.join(results_folder, f"{file_name[:-4]}_n.png"))),
                ]
                is_mask = False  # Input images do not need binary transformation
            elif subfolder == "labels":
                # 1 band: binary mask
                bands = [
                    np.array(Image.open(os.path.join(results_folder, f"{file_name[:-4]}_b.png"))),
                ]
                is_mask = True  # Apply binary transformation for masks

            # Pack the new GeoTIFF
            pack_geotiff(metadata, output_tiff_path, bands, scale_factor, is_mask)

print("GeoTIFF packing completed!")


Packing GeoTIFFs for train/input...


100%|██████████████████████████████████████████████████████████████████████████████| 1006/1006 [00:11<00:00, 89.40it/s]


Packing GeoTIFFs for train/labels...


100%|████████████████████████████████████████████████████████████████████████████████| 402/402 [00:05<00:00, 69.15it/s]


Packing GeoTIFFs for val/input...


...

Packing GeoTIFFs for test/input...


100%|████████████████████████████████████████████████████████████████████████████████| 180/180 [00:01<00:00, 93.61it/s]


Packing GeoTIFFs for test/labels...


100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 91.27it/s]

GeoTIFF packing completed!
